In [ ]:
import json
import altair as alt
import pandas as pd
import geopandas
import numpy as np
import polars as pl

alt.data_transformers.enable("vegafusion")

In [ ]:
with open("../data/chidata_census_tracts_boundaries.geojson", "r") as f:
    tract_boundaries_raw = json.load(f)

tract_boundaries = geopandas.GeoDataFrame.from_features(tract_boundaries_raw["features"])
tract_boundaries["geoid10"] = tract_boundaries["geoid10"].astype(int)

tract_boundaries

In [ ]:
tract_invest = pl.read_csv("../data/urbinst_tract_invest_data.csv")
tract_invest

In [ ]:
tract_invest_long = tract_invest.melt(id_vars=["tract_geoid", "majority_race", "poverty_cat", "community"], variable_name="invest_cat", value_name="percentile")
tract_invest_long

In [ ]:
tract_invest_long["invest_cat"].unique()

In [ ]:
tract_invest_long = tract_invest_long.with_columns(pl.when(
    pl.col("invest_cat") == "aggregate_pct")
    .then(pl.lit("Aggregate investment"))
    .when(pl.col("invest_cat") == "nonres_pct")
    .then(pl.lit("Non-residential loans (commercial, industrial, agricultural)"))
    .when(pl.col("invest_cat") == "small_biz_pct")
    .then(pl.lit("Small-business loans (less than $1M in revenue)"))
    .when(pl.col("invest_cat") == "mission_pct")
    .then(pl.lit("Community development funds (financial intitutions, misc. lenders)"))
    .when(pl.col("invest_cat") == "public_pct")
    .then(pl.lit("Federal community development funds"))
    .when(pl.col("invest_cat") == "SF_pct")
    .then(pl.lit("Purchase loans for owner-occupied single-family properties (1-4 units)"))
    .otherwise(pl.lit("Purchase loans for renter-occupied multi-family properties (5 or more units)"))
    .alias("invest_cat"))

tract_invest_long

In [ ]:
investment = list(tract_invest_long["invest_cat"].unique())

investment_radio = alt.binding_radio(options=investment, name="Investment Category ")
investment_option = alt.selection_point(fields=['invest_cat'], bind=investment_radio, value="Aggregate investment")

base = (
    alt.Chart(tract_boundaries).mark_geoshape(
        fill='lightgray',
        strokeWidth=1
    ).encode(
    ).project(type="mercator").properties(
        width=550,
        height=750,
        title="Average Scaled Investment by Tract in Chicago (2010-2020)"
    ) 
)

fig = (
    alt.Chart(tract_invest_long)
    .mark_geoshape(strokeWidth=1.2, stroke="white")
    .encode(
        shape="geo:G",
        color=alt.Color("percentile:Q", scale=alt.Scale(range=['#fed16f','#fea045','#fc7335','#ec3f2f','#b21218'], domain=[0, 20, 40, 60, 80, 100]), bin=True).title("Percentile"),
        tooltip=[alt.Tooltip('tract_geoid:N', title="Tract ID"), alt.Tooltip('community:N',title="Neighborhood"), alt.Tooltip('percentile:Q', title="Percentile")],
    )
    .transform_lookup(
        lookup="tract_geoid", 
        from_=alt.LookupData(data=tract_boundaries, key="geoid10"), 
        as_="geo"
    ).add_params(investment_option)
    .transform_filter(investment_option)
    .project(type="mercator")
    .properties(
        width=550,
        height=750
    )
)

In [ ]:
selection = alt.selection_point(on="click", fields=["community"], clear="dbclick")

fig_selection = fig.encode(
    strokeWidth=alt.condition(selection, alt.value(1.2), alt.value(0.1)),
    strokeDash=alt.when(selection).then(alt.value([0])).otherwise(alt.value([1, 1])),
    ).add_params(selection)

chicago_map = base + fig_selection

In [ ]:
tract_invest_long["majority_race"].unique()
tract_invest_long["poverty_cat"].unique()

In [ ]:
# Race Majority dropdown
race_select_options = [None, 'no racial majority', 'white', 'black', 'latine', 'asian']
race_label_options = ['all', 'no racial majority', 'white', 'black', 'latine', 'asian']

race_majority_dropdown = alt.binding_select(options=race_select_options, labels=race_label_options, name="Race Majority ")
race_majority_select = alt.selection_point(fields=["majority_race"], bind=race_majority_dropdown)

# Poverty Category dropdown
pov_select_options = [None, 'Over 31% Poverty', '21-31% Poverty', '12-20% Poverty', '0-11% Poverty']
pov_label_options = ['All', 'Over 31% Poverty', '21-31% Poverty', '12-20% Poverty', '0-11% Poverty']

pov_dropdown = alt.binding_select(options=pov_select_options, labels=pov_label_options, name="Poverty Category ")
pov_select = alt.selection_point(fields=["poverty_cat"], bind=pov_dropdown)

In [ ]:
final_map = chicago_map.add_params(race_majority_select, pov_select).encode(
    opacity=alt.condition(race_majority_select & pov_select, alt.value(1.0), alt.value(0.2)))

final_map

In [ ]:
# https://github.com/vega/altair/issues/2875
def export_viz(graph, filename):
    json_file = graph.to_json(format="vega")
    with open(f'../visualizations/{filename}.json', "w") as f:
        f.write(json_file)

export_viz(final_map, "tract_map")

In [ ]:
top_cities_raw = pl.read_csv("../data/urbinst_top_cities.csv")
state_region = pl.read_csv("../data/kaggle_state_region_crosswalk.csv")

def process_top_cities(num_cities=100):
    df = top_cities_raw.sort(by="place_pop", descending=True).head(num_cities)
    df = top_cities_raw.with_columns(pl.col("placenm").str.tail(2).alias("state"))
    df = df.join(state_region["State Code", "Region", "Division"], left_on="state", right_on="State Code", how="left").rename({"Region": "region", "Division": "division"})
    return df

top_cities = process_top_cities()

In [ ]:
regions = top_cities["region"].unique().to_list()

region_options = [None] + regions
region_labels = ["All"] + regions

region_dropdown = alt.binding_select(options=region_options, labels=region_labels, name="Region ")
region_select = alt.selection_point(fields=["region"], bind=region_dropdown)

Radio by Investment Type

In [ ]:
top_cities_long = top_cities.melt(id_vars=['place_geoid',
 'placenm',
 'place_pop',
 'poverty_perc',
 'state',
 'region',
 'division'
 ], variable_name="measure", value_name="inv_amount")

top_cities_long

In [ ]:
list(top_cities_long["measure"].unique())

In [ ]:
top_cities_long = top_cities_long.with_columns(pl.when(
    pl.col("measure") == "agg_investment_perhh")
    .then(pl.lit("Aggregate investment per Household"))
    .when(pl.col("measure") == "ACII_loans_peremp")
    .then(pl.lit("Non-residential investment per private employee"))
    .when(pl.col("measure") == "small_business_lending_persbemp")
    .then(pl.lit("Small-business loans per small business employee"))
    .when(pl.col("measure") == "mission_lending_perhh")
    .then(pl.lit("Community development funds per Household"))
    .when(pl.col("measure") == "federal_perhh")
    .then(pl.lit("Federal community development funds per Household"))
    .when(pl.col("measure") == "HMDA_SF_perSFhh")
    .then(pl.lit("Purchase loans per owner-occupied household"))
    .otherwise(pl.lit("Purchase loans per renter-occupied household"))
    .alias("measure"))

top_cities_long

In [ ]:
measures = top_cities_long["measure"].unique().to_list()

measures_radio = alt.binding_radio(options=measures, name="Investment Category ")
measures_option = alt.selection_point(fields=['measure'], bind=measures_radio, value="Aggregate investment per Household")

In [ ]:
interval = alt.selection_interval(encodings=['x'])

scatter2 = alt.Chart(top_cities_long).mark_point(filled=True
).encode(
    alt.Y("poverty_perc:Q", axis=alt.Axis(labelExpr="datum.value * 100 + '%'"), title="Poverty percentage"),
    alt.X("inv_amount:Q", title="Investment amount (USD, per scalar unit)"),
    alt.Size("place_pop:Q", scale=alt.Scale(domain=[350000, 8500000], range=[50, 1500]), title="Population", legend=alt.Legend(labelExpr="datum.value / 1000000 + 'M'")),
    alt.Color("region:N", title="Region"),
    tooltip=[alt.Tooltip("placenm", title="City"), alt.Tooltip("place_pop:Q", format=",", title="Population"), alt.Tooltip("poverty_perc:Q", format=".2%", title="Poverty percentage"), alt.Tooltip("inv_amount:Q", format=",.2f", title="Investment amount (USD)")]
).properties(
    width=500,
    height=375,
    title="Average Scaled Investment in the US's top 100 cities (2010-2020)"
)

hist3 = alt.Chart(top_cities_long).mark_bar().add_params(interval).transform_filter(interval).transform_aggregate(ct = 'count(measure)',
    groupby=['region', "measure"]).encode(
    alt.X('mean(ct):Q', title="City Count"),
    alt.Y('region:N', title="Region"),
    color='region:N').properties(
    width=500
)

In [ ]:
a = scatter2.encode(opacity=alt.condition(measures_option, alt.value(0.7), alt.value(0))).add_params(
    measures_option
)

b = hist3.add_params(
    measures_option
).transform_filter(measures_option)

select_region = alt.selection_point(fields=["region"], bind="legend")

c = a.encode(
    color=alt.condition(select_region, alt.Color("region:N", title="Region", scale=alt.Scale(range=["#7DB583", "#3B5268", "#FFC857", "#E07B7B"])), alt.value('#ECECEC')),
).properties(
    width=500,
    height=375
).add_params(select_region)

In [ ]:
# Ref: https://stackoverflow.com/questions/54964869/how-does-one-highlight-a-mark-in-a-scatter-plot-using-altair
text = alt.Chart(top_cities_long).mark_text(align="left", dx=2, dy=-17, color="maroon").encode(
    x=alt.X("inv_amount:Q"), 
    y=alt.Y("poverty_perc:Q"),
    text=alt.Text("placenm:N")
)

chi_text = text.transform_filter("datum.placenm == 'Chicago, IL'").add_params(
    measures_option
).transform_filter(measures_option)

border = c.transform_filter("datum.placenm == 'Chicago, IL'").mark_point().encode(
    color=alt.value("maroon")
)

chicago_highlight = border + chi_text
scatter_fin = c + chicago_highlight
scatter_fin

In [ ]:
final_cities_chart = scatter_fin.add_params(interval) & b.add_params(interval)
final_cities_chart

In [ ]:
export_viz(final_cities_chart, "top_cities")

In [ ]:
time_series = pl.read_csv("../data/urbinst_chicago_timeseries.csv")
time_series_long = time_series.melt(id_vars="year", variable_name="measure", value_name="inv_amount")
list(time_series_long["measure"].unique())

In [ ]:
time_series_long = time_series_long.with_columns(pl.when(
    pl.col("measure") == "agg_investment_perhh")
    .then(pl.lit("Aggregate investment"))
    .when(pl.col("measure") == "ACII_loans_peremp")
    .then(pl.lit("Non-residential investment"))
    .when(pl.col("measure") == "small_business_lending_persbemp")
    .then(pl.lit("Small-business loans"))
    .when(pl.col("measure") == "mission_lending_perhh")
    .then(pl.lit("Community development funds"))
    .when(pl.col("measure") == "federal_public_perhh")
    .then(pl.lit("Federal funds"))
    .when(pl.col("measure") == "HMDA_SF_perSFhh")
    .then(pl.lit("Purchase loans (SF units)"))
    .otherwise(pl.lit("Purchase loans (MF units)"))
    .alias("measure"))

time_series_long = time_series_long.with_columns(pl.col("inv_amount").cast(pl.Decimal(scale=2, precision=None)))

In [ ]:
# https://altair-viz.github.io/gallery/multiline_tooltip.html
line = alt.Chart(time_series_long).mark_line(strokeWidth=2).encode(
    alt.X("year:T", title="Year"),
    alt.Y("inv_amount:Q", title="Investment amount (USD, per scalar unit)"),
    alt.Color("measure", scale=alt.Scale(range=["#7DB583", "#3B5268", "#FFC857", "#E07B7B", "#7BAEE0", "#CE94E6", "#E08D67"]), title="Investment Category"),
    tooltip=[alt.Tooltip("year:T", format="%Y", title="Year"), alt.Tooltip("measure", title="Investment Category"), alt.Tooltip("inv_amount:N", format=",.2f", title="Amount (USD)")]
).properties(
    width=300,
    height=300,
    title="Chicago Investment trend (2010-2020)"
)

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection_point(nearest=True, on="click",
                              fields=["year"], empty=False)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(time_series_long).mark_point().encode(
    x="year:T",
    opacity=alt.value(0),
).add_params(
    nearest
)
when_near = alt.when(nearest)

# Draw points on the line, and highlight based on selection
points = line.mark_point(size=100, filled=True).encode(
    opacity=when_near.then(alt.value(1)).otherwise(alt.value(0))
)

# Draw a rule at the location of the selection
rules = alt.Chart(time_series_long).mark_rule(color="gray", strokeWidth=0.7).encode(
    x="year:T",
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
final_time_series_chart = alt.layer(
    line, selectors, rules, points,
)

final_time_series_chart

In [ ]:
export_viz(final_time_series_chart, "trend")

In [ ]:
categories = pl.read_csv("../data/urbinst_chicago_categories.csv").drop("agg_investment")
categories_long = categories.melt(variable_name="measure", value_name="inv_amount")
categories_long = categories_long.with_columns((pl.col("inv_amount") / 1000000000).cast(pl.Decimal(scale=2, precision=None)))
categories_long

In [ ]:
categories_long = categories_long.with_columns(pl.when(
    pl.col("measure") == "ACII_loans")
    .then(pl.lit("Non-residential investment"))
    .when(pl.col("measure") == "small_business_lending")
    .then(pl.lit("Small-business loans"))
    .when(pl.col("measure") == "mission_lending")
    .then(pl.lit("Community development funds"))
    .when(pl.col("measure") == "federal_public")
    .then(pl.lit("Federal funds"))
    .when(pl.col("measure") == "HMDA_SF")
    .then(pl.lit("Purchase loans (SF units)"))
    .otherwise(pl.lit("Purchase loans (MF units)"))
    .alias("measure"))

In [ ]:
final_donut_chart = alt.Chart(categories_long).mark_arc(innerRadius=90).transform_calculate(inv_amt="'$' + datum.inv_amount + 'B'").encode(
    theta="inv_amount:Q",
    color=alt.Color("measure:N", scale=alt.Scale(range=["#CE94E6", "#7DB583", "#E07B7B", "#FFC857", "#3B5268", "#7BAEE0"]), title="Investment Category"),
    tooltip=[alt.Tooltip("measure", title="Investment Type"), alt.Tooltip("inv_amt:N", title="Amount")]
).properties(
    width=300,
    height=300,
    title=alt.Title("Chicago Total Investment by Category (2010-2020)", anchor="middle")
)

final_donut_chart

In [ ]:
export_viz(final_donut_chart, "invest_categories")

In [ ]:
inv_byrace = pl.read_csv("../data/urbinst_chicago_race.csv")
inv_byrace = inv_byrace.drop("n_tracts")

In [ ]:
inv_byrace = inv_byrace.with_columns(pl.when(
    pl.col("race_cat").str.contains("white"))
    .then(pl.lit("White"))
    .when(pl.col("race_cat").str.contains("Black"))
    .then(pl.lit("Black"))
    .when(pl.col("race_cat").str.contains("Asian"))
    .then(pl.lit("Asian"))
    .when(pl.col("race_cat").str.contains("Latine"))
    .then(pl.lit("Latine"))
    .alias("race")
    )

inv_byrace

In [ ]:
inv_byrace_long = inv_byrace.melt(id_vars=["race", "race_cat"], variable_name="measure", value_name="inv_amount")
inv_byrace_long

In [ ]:
inv_byrace_long = inv_byrace_long.with_columns(pl.when(
    pl.col("race_cat") == "0-5% Asian")
    .then(pl.lit("(1) 0-5% Asian"))
    .when(pl.col("race_cat") == "6-10% Asian")
    .then(pl.lit("(2) 6-10% Asian"))
    .when(pl.col("race_cat") == "11-20% Asian")
    .then(pl.lit("(3) 11-20% Asian"))
    .when(pl.col("race_cat") == "21+% Asian")
    .then(pl.lit("(4) 21+% Asian"))
    .otherwise(pl.col("race_cat"))
    .alias("race_cat")
    )

list(inv_byrace_long["measure"].unique())

inv_byrace_long = inv_byrace_long.with_columns(pl.when(
    pl.col("measure") == "agg_investment_perhh")
    .then(pl.lit("Aggregate investment per Household"))
    .when(pl.col("measure") == "ACII_loans_peremp")
    .then(pl.lit("Non-residential investment per private employee"))
    .when(pl.col("measure") == "small_business_lending_persbemp")
    .then(pl.lit("Small-business loans per small business employee"))
    .when(pl.col("measure") == "mission_lending_perhh")
    .then(pl.lit("Community development funds per Household"))
    .when(pl.col("measure") == "federal_perhh")
    .then(pl.lit("Federal community development funds per Household"))
    .when(pl.col("measure") == "HMDA_SF_perSFhh")
    .then(pl.lit("Purchase loans per owner-occupied household"))
    .otherwise(pl.lit("Purchase loans per renter-occupied household"))
    .alias("measure"))

In [ ]:
inv = list(inv_byrace_long["measure"].unique())

inv_radio = alt.binding_radio(options=inv, name="Investment Category ")
inv_option = alt.selection_point(fields=['measure'], bind=inv_radio, value="Aggregate investment per Household")

inv_byrace_final_chart = alt.Chart(inv_byrace_long).mark_bar(color="maroon").encode(
    alt.X("inv_amount:Q", title="Investment Amount (USD, per scalar unit)"),
    alt.Y("race_cat:N", title=None),
    tooltip=[alt.Tooltip("inv_amount", title="Investment Amount (USD)", format=",.2f")]
    ).facet(
        row=alt.Row("race", title=None),
    ).resolve_scale(y='independent').add_params(inv_option).transform_filter(inv_option).properties(
        title="Chicago Average Scaled Investment by Tracts with Race Majority (2010-2020)"
    )

inv_byrace_final_chart

In [ ]:
export_viz(inv_byrace_final_chart, "investment_byrace")

In [ ]:
inv_bypov = pl.read_csv("../data/urbinst_chicago_poverty.csv")
inv_bypov = inv_bypov.drop("n_tracts")

inv_bypov

In [ ]:
inv_bypov_long = inv_bypov.melt(id_vars=["poverty_cat"], variable_name="measure", value_name="inv_amount")
list(inv_bypov_long["measure"].unique())

inv_bypov_long = inv_bypov_long.with_columns(pl.when(
    pl.col("measure") == "agg_investment_perhh")
    .then(pl.lit("Aggregate investment per Household"))
    .when(pl.col("measure") == "ACII_loans_peremp")
    .then(pl.lit("Non-residential investment per private employee"))
    .when(pl.col("measure") == "small_business_lending_persbemp")
    .then(pl.lit("Small-business loans per small business employee"))
    .when(pl.col("measure") == "mission_lending_perhh")
    .then(pl.lit("Community development funds per Household"))
    .when(pl.col("measure") == "federal_public_perhh")
    .then(pl.lit("Federal community development funds per Household"))
    .when(pl.col("measure") == "HMDA_SF_perSFhh")
    .then(pl.lit("Purchase loans per owner-occupied household"))
    .otherwise(pl.lit("Purchase loans per renter-occupied household"))
    .alias("measure"))

In [ ]:
inv_bypov_final_chart = alt.Chart(inv_bypov_long).mark_bar(color="maroon").encode(
    alt.X("inv_amount:Q", title="Investment Amount (USD, per scalar unit)"),
    alt.Y("poverty_cat:N", title=None),
    tooltip=[alt.Tooltip("inv_amount", title="Investment Amount (USD)", format=",.2f")],
    facet=alt.Facet("measure", title=None, columns=2, sort=["Purchase loans per renter-occupied household", "Purchase loans per owner-occupied household", "Non-residential investment per private employee", "Small-business loans per small business employee", "Federal community development funds per Household", "Community development funds per Household", "Aggregate investment per Household"])
    ).properties(
        title=alt.Title("Chicago Average Scaled Investment by Tracts with Poverty Level (2010-2020)", anchor="middle")
    )

inv_bypov_final_chart

In [ ]:
export_viz(inv_bypov_final_chart, "investment_bypoverty")